# SVM Classifier for Hard Negative Mining Project


In [22]:
# Run some setup code for this notebook.
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
#import lr_scheduler
import torch.optim.lr_scheduler

from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import operator
import numpy as np
import cv2
import matplotlib.pyplot as plt

from skimage.feature import hog
from skimage import data, exposure

# This is to make matplotlib figures appear inline in the notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# So the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def applySIFT(img):
#     gray = np.uint8(image.numpy()[0, 0, :, :]) # grayscale image
    print(img.shape)
    gray= cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    sift = cv2.SIFT()
    kp = sift.detect(gray,None)
#     sift = cv2.xfeatures2d.SIFT_create()
#     keypoints = sift.detect(gray, None)
    return kp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from os import listdir
from PIL import Image as PImage

def loadImages(path):
    # return array of images

    imagesList = listdir(path)
    print(imagesList)
    loadedImages = []
    for image in imagesList:
        img = PImage.open(os.path.join(path, image))
        loadedImages.append(img)

    return loadedImages

path = os.path.join('facesDataAll', 'train') #"/path/to/your/images/"
print(path)
# your images in an array
imgs = loadImages(path)

for img in imgs:
    # you can show every image
    img.show()

facesDataAll/train
['.DS_Store', 'attractive', 'unattractive']


IOError: cannot identify image file 'facesDataAll/train/.DS_Store'

## SIFT feature processing


In [23]:
## Run HOG feature extraction on all images and use as our data

data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(256),
        transforms.RandomCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'dev': transforms.Compose([
        transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
    transforms.Scale(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'facesDataAll'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'dev', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                              shuffle=True, num_workers=0)
               for x in ['train', 'dev', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'dev', 'test']}

class_names = image_datasets['train'].classes

# Iterate over data.
X_train = []
Y_train = []
for data in dataloaders['train']:
    # get the inputs
    image, label = data
#     newImage = image.numpy()[0, 0, :, :]
#     image = image.numpy()
    newLabel = label.numpy()
    image = image.numpy()
    print(image.shape)
    image.reshape(3, image.shape[2], image.shape[3])
    print(image.shape)


    keypoints = applySIFT(image)
    X_train.append(keypoints)
    Y_train.append(newLabel[0])
print('Done processing train set')

X_dev = []
Y_dev = []
for data in dataloaders['dev']:
    # get the inputs
    image, label = data
    newLabel = label.numpy()
#     newImage = image.numpy()[0, 0, :, :]
    keypoints = applySIFT(image)
    X_dev.append(keypoints)
    Y_dev.append(newLabel[0])

print('Done processing dev set')
X_test = []
Y_test = []
for data in dataloaders['test']:
    # get the inputs
    image, label = data
    newLabel = label.numpy()
#     newImage = image.numpy()[0, 0, :, :]
    keypoints = applySIFT(image)
    X_test.append(keypoints)
    Y_test.append(newLabel[0])

print('Done processing test set')


(1, 3, 224, 224)
(1, 3, 224, 224)
(1, 3, 224, 224)


error: /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/color.cpp:10638: error: (-215) scn == 3 || scn == 4 in function cvtColor


In [3]:
X_test = np.array(X_test)
y_test = np.array(Y_test)

X_train = np.array(X_train)
y_train = np.array(Y_train)

X_dev = np.array(X_dev)
y_dev = np.array(Y_dev)

In [4]:


# Flatten and convert data into rows as part of out preprocess
X_train = np.reshape(X_train, (X_train.shape[0], -1))
X_test = np.reshape(X_test, (X_test.shape[0], -1))
X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))

#reshape y's
y_train = np.reshape(y_train, (X_train.shape[0], 1))
y_test = np.reshape(y_test, (X_test.shape[0], 1))
y_dev = np.reshape(y_dev, (X_dev.shape[0], 1))

#Sanity Check
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)

print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)


print('Dev data shape: ', X_dev.shape)
print('Dev labels shape: ', y_dev.shape)

Train data shape:  (948, 1)
Train labels shape:  (948, 1)
Test data shape:  (183, 1)
Test labels shape:  (183, 1)
Dev data shape:  (180, 1)
Dev labels shape:  (180, 1)


In [5]:
# Normalizing
mean_image = np.mean(X_train, axis=0)

X_train -= mean_image
# X_val -= mean_image
X_test -= mean_image
X_dev -= mean_image

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [ ]:
# Add bias
X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
# X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])

# print(X_train.shape, X_val.shape, X_test.shape, X_dev.shape)
print(X_train.shape, X_test.shape, X_dev.shape)

### SVM

Compute SVM to minimize the loss.

In [ ]:
# from utils.classifiers import LinearSVM
# svm = LinearSVM()
# tic = time.time()
# loss_hist = svm.train(X_train, y_train, learning_rate=1e-7, reg=2.5e4,
#                       num_iters=1500, verbose=True)
# toc = time.time()
# print('That took %fs' % (toc - tic))

from sklearn.svm import SVC
clf = SVC()
tic = time.time()
clf.fit(X_train, y_train.ravel()) 
toc = time.time()
print('That took %fs' % (toc - tic))


test_acc = clf.score(X_test, y_test.ravel())
train_acc = clf.score(X_train, y_train.ravel())
print('Train acc: ', train_acc)
print('Test acc: ', test_acc)